In [2]:
%pip install -q -U torch==2.0.1 bitsandbytes==0.40.2
%pip install -q -U transformers==4.35.2 peft==0.4.0 accelerate==0.21.0
%pip install -q -U datasets py7zr einops tensorboardX
!pip install evaluate
# Add installed cuda runtime to path for bitsandbytes
import os
import nvidia

cuda_install_dir = '/'.join(nvidia.__file__.split('/')[:-1]) + '/cuda_runtime/lib/'
os.environ['LD_LIBRARY_PATH'] =  cuda_install_dir


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip ins

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import pandas as pd
import os
import torch
import numpy as np
from scipy import stats
# Path to your env.txt file
env_file_path = 'data/env.txt'

# Read and set environment variables
with open(env_file_path, 'r') as file:
    for line in file:
        key, value = line.strip().split('=')
        os.environ[key] = value
token = os.environ['huggingface_token']

In [16]:
model_path = "meta-llama/Llama-2-70b-hf"   # Specify the path to the model
tokenizer = AutoTokenizer.from_pretrained(model_path, token=token)
# model = AutoModelForCausalLM.from_pretrained(
#   model_path,
#   device_map='auto',
#   load_in_4bit=True,
#   do_sample=True,
#   torch_dtype="auto", 
#     token=token,
# )
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    load_in_4bit=True,    # changing this to load_in_8bit=True works on smaller models
    trust_remote_code=True,
    token=token,
    device_map="auto",    # finds GPU
)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [18]:
df = pd.read_csv('data/all_models.csv')
gsm8k_all = pd.read_csv('data/gsm8k_all.csv')
gsm8k_questions = pd.read_csv('data/gsm8k_questions.csv')
mathwell_all = pd.read_csv('data/mathwell_annotations.csv')
mathwell_all_good = mathwell_all[mathwell_all['good']==1]
llama = df[df['model']=='llama']
llama_good = llama[llama['good']==1]
llema = df[df['model']=='llema']
llema_good = llema[llema['good']==1]
mathwell = df[df['model']=='mathwell']
mathwell_good = mathwell[mathwell['good']==1]
mammoth = df[df['model']=='mammoth']
mammoth_good = mammoth[mammoth['good']==1]

In [19]:
def perplexity(df):
    ppls = []
    for i in range(0, len(df)):
        text = "Question: " + df.iloc[i]['question'] + "\n" + "Solution:\n" + df.iloc[i]['solution']
        inputs = tokenizer(text, return_tensors = "pt")
        loss = model(input_ids = inputs["input_ids"], labels = inputs["input_ids"]).loss
        ppl = torch.exp(loss)
        ppl = ppl.cpu().detach().numpy()
        ppls.append(ppl)
    return ppls

def perplexity_question(df):
    ppls = []
    for i in range(0, len(df)):
        text = df.iloc[i]['question']
        inputs = tokenizer(text, return_tensors = "pt")
        loss = model(input_ids = inputs["input_ids"], labels = inputs["input_ids"]).loss
        ppl = torch.exp(loss)
        ppl = ppl.cpu().detach().numpy()
        ppls.append(ppl)
    return ppls

def perplexity_gsm(df):
    ppls = []
    for i in range(0, len(df)):
        text = df.iloc[i]['output']
        inputs = tokenizer(text, return_tensors = "pt")
        loss = model(input_ids = inputs["input_ids"], labels = inputs["input_ids"]).loss
        ppl = torch.exp(loss)
        ppl = ppl.cpu().detach().numpy()
        ppls.append(ppl)
    return ppls

def perplexity_gsm_question(df):
    ppls = []
    for i in range(0, len(df)):
        text = df.iloc[i]['instruction']
        inputs = tokenizer(text, return_tensors = "pt")
        loss = model(input_ids = inputs["input_ids"], labels = inputs["input_ids"]).loss
        ppl = torch.exp(loss)
        ppl = ppl.cpu().detach().numpy()
        ppls.append(ppl)
    return ppls

## GSM8K Perplexity

In [24]:
gsm8k_ppl = perplexity_gsm(gsm8k_all)
gsm8k_question_ppl = perplexity_gsm_question(gsm8k_questions)
print(f'Average GSM8K overall perplexity: {np.mean(gsm8k_ppl)} Standard Deviation: {np.std(gsm8k_ppl)}')
print(f'Average GSM8K overall perplexity for questions only: {np.mean(gsm8k_question_ppl)} Standard Deviation: {np.std(gsm8k_question_ppl)}')

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 0; 14.76 GiB total capacity; 13.74 GiB already allocated; 99.75 MiB free; 13.98 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## MATHWELL Annotated Perplexity

In [ ]:
mathwell_all_ppl = perplexity(mathwell_all)
mathwell_all_question_ppl = perplexity_question(mathwell_all)
print(f'Average MATHWELL Annotated overall perplexity: {np.mean(mathwell_all_ppl)} Standard Deviation: {np.std(mathwell_all_ppl)}')
print(f'Average MATHWELL Annotated overall perplexity for questions only: {np.mean(mathwell_all_question_ppl)} Standard Deviation: {np.std(mathwell_all_question_ppl)}')

In [ ]:
mathwell_all_good_ppl = perplexity(mathwell_all_good)
mathwell_all_good_question_ppl = perplexity_question(mathwell_all_good)
print(f'Average MATHWELL Train overall perplexity: {np.mean(mathwell_all_good_ppl)} Standard Deviation: {np.std(mathwell_all_good_ppl)}')
print(f'Average MATHWELL Train overall perplexity for questions only: {np.mean(mathwell_all_good_question_ppl)} Standard Deviation: {np.std(mathwell_all_good_question_ppl)}')

## MATHWELL Final Perplexity

In [ ]:
mathwell_ppl = perplexity(mathwell)
mathwell_question_ppl = perplexity_question(mathwell)
print(f'Average MATHWELL overall perplexity: {np.mean(mathwell_ppl)} Standard Deviation: {np.std(mathwell_ppl)}')
print(f'Average MATHWELL overall perplexity for questions only: {np.mean(mathwell_question_ppl)} Standard Deviation: {np.std(mathwell_question_ppl)}')

In [ ]:
mathwell_good_ppl = perplexity(mathwell_good)
mathwell_good_question_ppl = perplexity_question(mathwell_good)
print(f'Average MATHWELL Good overall perplexity: {np.mean(mathwell_good_ppl)} Standard Deviation: {np.std(mathwell_good_ppl)}')
print(f'Average MATHWELL Good overall perplexity for questions only: {np.mean(mathwell_good_question_ppl)} Standard Deviation: {np.std(mathwell_good_question_ppl)}')

## Llama Perplexity

In [ ]:
llama_ppl = perplexity(llama)
llama_question_ppl = perplexity_question(llama)
print(f'Average Llama overall perplexity: {np.mean(llama_ppl)} Standard Deviation: {np.std(llama_ppl)}')
print(f'Average Llama overall perplexity for questions only: {np.mean(llama_question_ppl)} Standard Deviation: {np.std(llama_question_ppl)}')

In [ ]:
llama_good_ppl = perplexity(llama_good)
llama_good_question_ppl = perplexity_question(llama_good)
print(f'Average Llama Good overall perplexity: {np.mean(llama_good_ppl)} Standard Deviation: {np.std(llama_good_ppl)}')
print(f'Average Llama Good overall perplexity for questions only: {np.mean(llama_good_question_ppl)} Standard Deviation: {np.std(llama_good_question_ppl)}')

## Llemma Perplexity

In [ ]:
llema_ppl = perplexity(llema)
llema_question_ppl = perplexity_question(llema)
print(f'Average Llemma overall perplexity: {np.mean(llema_ppl)} Standard Deviation: {np.std(llema_ppl)}')
print(f'Average Llemma overall perplexity for questions only: {np.mean(llema_question_ppl)} Standard Deviation: {np.std(llema_question_ppl)}')

In [ ]:
llema_good_ppl = perplexity(llema_good)
llema_good_question_ppl = perplexity_question(llema_good)
print(f'Average Llemma Good overall perplexity: {np.mean(llema_good_ppl)} Standard Deviation: {np.std(llema_good_ppl)}')
print(f'Average Llemma Good overall perplexity for questions only: {np.mean(llema_good_question_ppl)} Standard Deviation: {np.std(llema_good_question_ppl)}')

## Mammoth Perplexity

In [ ]:
mammoth_ppl = perplexity(mammoth)
mammoth_question_ppl = perplexity_question(mammoth)
print(f'Average Mammoth overall perplexity: {np.mean(mammoth_ppl)} Standard Deviation: {np.std(mammoth_ppl)}')
print(f'Average Mammoth overall perplexity for questions only: {np.mean(mammoth_question_ppl)} Standard Deviation: {np.std(mammoth_question_ppl)}')

In [ ]:
mammoth_good_ppl = perplexity(mammoth_good)
mammoth_good_question_ppl = perplexity_question(mammoth_good)
print(f'Average Mammoth Good overall perplexity: {np.mean(mammoth_good_ppl)} Standard Deviation: {np.std(mammoth_good_ppl)}')
print(f'Average Mammoth Good overall perplexity for questions only: {np.mean(mammoth_good_question_ppl)} Standard Deviation: {np.std(mammoth_good_question_ppl)}')